In [19]:
#import package
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm

#load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

x1 = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
x1 = pd.get_dummies(x1)
x2 = train['Exited']
#切割訓練資料
x1_train, x1_test, x2_train, x2_test = train_test_split(x1,x2, train_size=0.8,random_state=3)

#標準化
scaler = StandardScaler()
scaler.fit(x1_train)
x1_train_std = scaler.transform(x1_train)
x1_test_std = scaler.transform(x1_test)
#套模型
svc = svm.SVC()
svc.fit(x1_train_std, x2_train)

x1_test_predict = svc.predict(x1_test_std)
# test accuracy score
np.mean(x1_test_predict==x2_test)

y = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'Tenure', 'NumOfProducts',  'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
y = pd.get_dummies(y)

scaler.fit(x1)
x1_std = scaler.transform(x1)
svc.fit(x1_std, x2)

y_std = scaler.transform(y)
y_predict = svc.predict(y_std)
y_predict

df = pd.DataFrame({'RowNumber': test['RowNumber'], 'Exited': list(y_predict)})
df.to_csv(r'./predict_11.csv',columns=['RowNumber','Exited'],index=True,sep=',')

In [22]:
#import package
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

#test
data=pd.read_csv("train.csv")
train_X=data.iloc[:,0:-1]
train_y=data.iloc[:,-1]
test_X=pd.read_csv("test.csv")
#data preprocessing

#標準化
sc=StandardScaler()
ad_train_X=train_X.iloc[:,[3,6,7,8,9,12]]
ad_test_X=test_X.iloc[:,[3,6,7,8,9,12]]
ad1_train_X=sc.fit_transform(ad_train_X)
ad1_test_X=sc.transform(ad_test_X)
#重新編碼
ohe=OneHotEncoder()
ad2_train_X=train_X.iloc[:,[4,5,10,11]]
ad2_test_X=test_X.iloc[:,[4,5,10,11]]
ad3_train_X=ohe.fit_transform(ad2_train_X).toarray()
ad3_test_X=ohe.transform(ad2_test_X).toarray()

df1=pd.DataFrame(ad1_train_X)
df2=pd.DataFrame(ad3_train_X)
for i in range(6,15):
    df1[i]=df2[i-6]
df3=pd.DataFrame(ad1_test_X)
df4=pd.DataFrame(ad3_test_X)
for i in range(6,15):
    df3[i]=df4[i-6]
  
 #去掉不需要的欄位
df1.drop(columns=[9,11,13],inplace=True)
df3.drop(columns=[9,11,13],inplace=True)

adj_train_X=df1
adj_test_X=df3

clf=svm.SVC(gamma=0.15)
clf.fit(adj_train_X,train_y)
pred_y=clf.predict(adj_test_X)

#create csv
outcome={"RowNumber":test_X["RowNumber"],
         "Exited":pred_y}
df_outcome=pd.DataFrame(outcome)
df_outcome.to_csv("predict_12.csv")